In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
df = pd.read_csv('../input/train.csv')
df_test = pd.read_csv('../input/test.csv')

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
grid = sns.FacetGrid(df, col = 'season', row = 'holiday')
grid.map(plt.scatter, 'temp', 'casual', alpha = 0.5)
grid.add_legend();

In [ ]:
plt.scatter(x = df['windspeed'], y = df['count'], alpha= .5)
plt.show()

In [ ]:
df[['count', 'holiday']].groupby(['holiday'], as_index = False).mean().sort_values(by = 'count')

In [ ]:
df[['count', 'workingday']].groupby(['workingday'], as_index = False).mean().sort_values(by = 'count')

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn import model_selection, preprocessing

In [ ]:
df.isnull().sum()

In [ ]:
df.datetime = df.datetime.apply(pd.to_datetime)
df['month'] = df.datetime.apply(lambda x: x.month)
df['hour'] = df.datetime.apply(lambda x: x.hour)
df['day'] = df.datetime.apply(lambda x: x.day)
#df['year'] = df.datetime.apply(lambda x: x.year)
df.drop(['datetime'], 1, inplace = True)
df.head()

** It is necessary to drop not only the label Count, but also the features Casual and Registered, as the sum of those will give you directly the value of Count, as Vivek Srinivasan
kindly pointed out. **

In [ ]:
plt.scatter(x = df['casual'] + df['registered'], y = df['count'])
plt.show()


In [ ]:
X = np.array(df.drop(df[['casual', 'registered', 'count']], 1))
y = np.array(df['count'])

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = 0.2)

We take the rmsle formula from Small Guo Jia's kernel, which is awesome (Thank you).

In [ ]:
def rmsle(y, y_):
    log1 = np.nan_to_num(np.array([np.log(v + 1) for v in y]))
    log2 = np.nan_to_num(np.array([np.log(v + 1) for v in y_]))
    calc = (log1 - log2) ** 2
    return np.sqrt(np.mean(calc))

In [ ]:
clf = LinearRegression(normalize= True)
clf.fit(X_train, y_train)
print("The accuracy of linear regression is: ", clf.score(X_test, y_test), "\nThe RMSLE is: ", rmsle(y_test, clf.predict(X_test)))

In [ ]:
clf = DecisionTreeRegressor()
clf.fit(X_train, y_train)
print("The accuracy of Decision Tree Regressor is: ", clf.score(X_test, y_test), "\nThe RMSLE is: ", rmsle(y_test, clf.predict(X_test)))

In [ ]:
clf = RandomForestRegressor(n_estimators= 200)
clf.fit(X_train, y_train)
print('The accuracy of Random Forest Regressor is: ', clf.score(X_test, y_test), "\nThe RMSLE is: ", rmsle(y_test, clf.predict(X_test)))

**We take our Random Forest Regression as the model we want to use, with a RMSLE of 41.904% **
